Importing necessary modules.

In [2]:
import numpy as np
import pandas as pd
import torch

Reading csv file.

In [3]:
data=pd.read_csv('2015.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441456 entries, 0 to 441455
Columns: 330 entries, _STATE to _AIDTST3
dtypes: float64(323), object(7)
memory usage: 1.1+ GB


Selecting columns from data that are relevant plus target column.

In [ ]:

columns_to_add=["BPHIGH4", "TOLDHI2", "CVDINFR4", "CVDCRHD4", "CVDSTRK3", "ASTHMA3", "ASTHNOW", "CHCSCNCR", "CHCOCNCR", "CHCCOPD1", "HAVARTH3", "ADDEPEV2",
                "CHCKIDNY", "SEX", "USENOW3", "FTJUDA1_", "FRUTDA1_", "BEANDAY_", "GRENDAY_", "_STATE","ORNGDAY_", "VEGEDA1_", "EXERANY2",
                "EXRACT11", "EXERHMM1", "STRENGTH", "ADEAT1", "_AGEG5YR", "_BMI5", "_SMOKER3", "DROCDY3_", "_TOTINDA", "PA1MIN_", "DIABETE3"]

data=data.filter(columns_to_add, axis=1)
data.head()
data.info()

Cleaning data and fixing format issues(null values, irrelevant values) and selecting certain US states from the data for easier processing.

In [ ]:

list1=["TOLDHI2", "CVDINFR4", "CVDCRHD4", "CVDSTRK3", "ASTHMA3", "ASTHNOW", "CHCSCNCR", "CHCOCNCR", "CHCCOPD1", "HAVARTH3", "ADDEPEV2",
                "CHCKIDNY"]
list2=[ "EXERANY2","EXRACT11", "EXERHMM1", "STRENGTH", "ADEAT1", "_TOTINDA", "BPHIGH4","TOLDHI2", "CVDINFR4", "CVDCRHD4",
        "CVDSTRK3", "ASTHMA3", "ASTHNOW", "CHCSCNCR", "CHCOCNCR", "CHCCOPD1", "HAVARTH3", "ADDEPEV2",
        "CHCKIDNY", "SEX" ]
list3=["FTJUDA1_", "FRUTDA1_", "BEANDAY_", "GRENDAY_", "ORNGDAY_", "VEGEDA1_", "PA1MIN_", "_BMI5"]

rows_to_drop=[]


data[["BPHIGH4", "DIABETE3"]]=data[["BPHIGH4", "DIABETE3"]].replace([2,4], 3)
data["DIABETE3"]=data["DIABETE3"].replace(3,0)
data[list1]=data[list1].replace([7,9,np.nan], 2)
data["USENOW3"]=data["USENOW3"].replace([7,9,np.nan], 3)
data[list3]=data[list3].fillna(data[list3].mean())
data["_SMOKER3"]=data["_SMOKER3"].replace(9,4)
data["DROCDY3_"]=data["DROCDY3_"].replace(900,0)

mask = data[["BPHIGH4", "DIABETE3"]].isna() | data[["BPHIGH4", "DIABETE3"]].isin([7,9])
mask2= data["_STATE"].isin( [2, 6, 8, 12, 42, 55, 22])

rows_to_drop=data.index[mask.any(axis=1)].tolist()
rows_to_drop+=data.index[data["_AGEG5YR"].isin([14])].tolist()
rows_to_drop+=data.index[~mask2].tolist()

rows_to_drop=list(set(rows_to_drop))
data.drop(labels=rows_to_drop, axis=0, inplace=True)
data.drop(labels="_STATE", axis=1)


clean_data=pd.get_dummies(data, columns=list2)

clean_data.info(verbose=True, show_counts=True)
    

Splitting the data in training and testing sets and conversion to torch tensors. Also administered the data to DataLoaders.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

y=clean_data["DIABETE3"]
X=clean_data.drop(labels="DIABETE3", axis=1)

X_train, X_test, y_train, y_test=train_test_split(X,y, train_size=.8, test_size=.2, random_state=1)

X_train, X_test, y_train, y_test=X_train.astype(float), X_test.astype(float), y_train.astype(float), y_test.astype(float)

X_train_tensor=torch.tensor(X_train.to_numpy(), dtype=torch.float)
X_test_tensor=torch.tensor(X_test.to_numpy(), dtype=torch.float)

y_train_tensor=torch.tensor(y_train.to_numpy(), dtype=torch.float).view(-1, 1)
y_test_tensor=torch.tensor(y_test.to_numpy(), dtype=torch.float).view(-1, 1)

train_dataset=TensorDataset(X_train_tensor, y_train_tensor)
test_dataset=TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader=DataLoader(train_dataset, batch_size=40, shuffle=True)
test_dataloader=DataLoader(test_dataset, batch_size=50, shuffle=False)





Defining the model.

In [13]:
import torch.nn as nn

class NNCalc(nn.Module):

    def __init__(self):
        super(NNCalc, self).__init__()
        self.fc1=nn.Linear(306, 128)
        self.fc2=nn.Linear(128, 64)
        self.fc3=nn.Linear(64, 32)
        self.fcf=nn.Linear(32, 1)
        self.relu=nn.ReLU()
        #self.sigmoid=nn.Sigmoid()
    

    def forward(self, x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        x=self.relu(x)
        x=self.fc3(x)
        x=self.relu(x)
        x=self.fcf(x)
        

        return x

model_NN=NNCalc()


Initializing loss and optimizer. The loss is being initialized with a biased weight for the positive labels, to reflect the imbalance in the data.

In [23]:


loss=nn.BCEWithLogitsLoss(pos_weight=torch.tensor([7.78]))
optimizer=torch.optim.Adam(model_NN.parameters(), lr=.001)

Training.

In [ ]:


epoch_num=1000

model_NN.train()


for nr in range(epoch_num):
    total_loss=0
    for values, labels in train_dataloader:
        optimizer.zero_grad()
        output=model_NN(values)
        BCELoss=loss(output,labels)
        BCELoss.backward()
        optimizer.step()
        total_loss+=BCELoss.item()
    if((nr+1)%100==0):
        print("The loss is",total_loss/len(train_dataloader))
           
    



Evaluating.

In [ ]:
model_NN.eval()
total_loss=0
predictions=[]
with torch.no_grad():
    for values, labels in test_dataloader:
        out=model_NN(values)
        loss_test=loss(out,labels)
        preds=(torch.sigmoid(out)>.5).int()
        total_loss+=loss_test.item()
        predictions.extend(preds)
    print("Test loss is ", total_loss/len(test_dataloader))


Final report.

In [38]:
from sklearn.metrics import classification_report, accuracy_score

acc=accuracy_score(y_test, predictions)
report=classification_report(y_test, predictions)

print("Accuracy is ", acc)
print("Report is\n", report)

Accuracy is  0.7117801283789893
Report is
               precision    recall  f1-score   support

         0.0       0.96      0.70      0.81      9802
         1.0       0.25      0.78      0.38      1259

    accuracy                           0.71     11061
   macro avg       0.61      0.74      0.60     11061
weighted avg       0.88      0.71      0.76     11061



The results are the following:

Accuracy is  0.7117801283789893
Report is
               precision    recall  f1-score   support

         0.0       0.96      0.70      0.81      9802
         1.0       0.25      0.78      0.38      1259

    accuracy                           0.71     11061
   macro avg       0.61      0.74      0.60     11061
weighted avg       0.88      0.71      0.76     11061

The results could be interpreted as far from ideal, but in my personal consideration the recall for the positive values, which is imperative for this model, is reasonably high(78%).